In [46]:
import os
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

In [47]:
select_code_list_intra = ["00001", "00010", "00011", "00100", "00101", 
                          "00110", "01000", "01001", "01010", 
                          "01100"]
select_code_list_inter = [ 
                         "00111",  
                          "01011", "01101", "01110", "01111"]

In [48]:
def rgb2gray(rgb):
    return np.dot(rgb[...,:3], [0.299, 0.587, 0.114])

In [49]:
def swap_landmark(base_, base_file, add_, add_file, i, dict_):
    coord_base = dict_[base_file][i - 1]
    coord_add = dict_[add_file][i - 1]
    base_width = coord_base[1][0] - coord_base[0][0] + 1
    base_height = coord_base[1][1] - coord_base[0][1] + 1
    add_crop = add_[coord_add[0][1] : coord_add[1][1] + 1, coord_add[0][0] : coord_add[1][0] + 1]
  
    
    
    rescaled = add_crop.astype(np.uint8)
   
    
    im = Image.fromarray(rescaled)
    im.save('temp.png')
    add_crop = Image.open('temp.png')
    img = add_crop.resize((base_width, base_height),Image.ANTIALIAS)
    img.save('temp.png')
    img = mpimg.imread('temp.png')
    base_[coord_base[0][1] : coord_base[1][1] + 1, coord_base[0][0] : coord_base[1][0] + 1] = img * 255.0
    
    return base_

In [50]:
def crop_to_augument(file_1, file_2, select_code, dict_):    
    img_1 = mpimg.imread(file_1)     
    gray_1 = img_1 * 255.0
    img_2 = mpimg.imread(file_2)     
    gray_2 = img_2 * 255.0
    if select_code[0] == "0":
        base_file = file_1
        base_ = gray_1
        add_file = file_2
        add_ = gray_2
    else:
        base_file = file_2
        base_ = gray_2
        add_file = file_1
        add_ = gray_1
    for i in range(1, len(select_code)):
        if select_code[i] != select_code[0]:
            base_ = swap_landmark(base_, base_file, add_, add_file, i, dict_)
    return base_

In [51]:
import numpy as np
dict_img_landmark = {}
dict_person_img = {}
landmark_file = "./landmark_full.txt"
data_path = "./dataset/lfw_mtcnnpy_gray_picked_10_8"
def generate_dict(dict_img_landmark, dict_person_img, landmark_file):
    file_ = open(landmark_file, "r")
    for line in file_: 
        list_ = line.strip().split(";")
        temp = list_[0]
        image_path = temp
        person = image_path.split("/")[0]
      
        
        temp = list_[1].replace(" ", "")
        temp = temp.replace("(", "")
        temp = temp.replace(")", "")
        temp = temp.replace("[", "")
        temp = temp.replace("]", "")
        coord_list = temp.split(",")
        coord_list = [int(int(ele) / 500.0 * 160.0) for ele in coord_list]
        if len(coord_list) != 16:
            continue
        
        coord_np = np.asarray(coord_list)
        coord_np = coord_np.reshape((4,2,2))
        
        if person not in dict_person_img:
            dict_person_img[person] = []
            
        dict_person_img[person].append(data_path + "/" + image_path)
        
        
        #print coord_np
        dict_img_landmark[data_path + "/" + image_path] = coord_np

In [52]:
generate_dict(dict_img_landmark, dict_person_img, landmark_file)

In [53]:
print len(dict_img_landmark.keys())

12544


In [54]:
import random 
person_picked = []
for folder in os.listdir(data_path):
    person_picked.append(folder)
visited = []
person_list = dict_person_img.keys()
cnt_augument = 0
flag = 0
while True:
    person_1 = person_picked[random.randint(0, len(person_picked) - 1)]
    person_2 = person_picked[random.randint(0, len(person_picked) - 1)]
    if person_1 != person_2:
        continue
    if [person_1, person_2] in visited:
        continue
    else:
        visited.append([person_1, person_2])
    
    print "visited: ", len(visited)
    #intra-syn
    if person_1 == person_2:
        #condition for num of pics, pending...
        image_list = dict_person_img[person_1]
        if len(image_list) <= 1 or len(image_list) >= 100:
            continue
        cnt_this_person = len(image_list)

        try:
            for i in range(len(image_list) - 1):
                for j in range(i + 1, len(image_list)):
                    image_1 = image_list[i]
                    image_2 = image_list[j]
                    for select_code in select_code_list_intra:
                        new_ = crop_to_augument(image_1, image_2, select_code, dict_img_landmark)
    #                         plt.imshow(new_, cmap = plt.get_cmap('gray'))
    #                         plt.show()
                        rescaled = new_.astype(np.uint8)
                        im = Image.fromarray(rescaled)
                        image_1_name = image_1.split("/")[4].split(".")[0]
                        image_2_name = image_2.split("/")[4].split(".")[0]
                        new_image_path = data_path + "/" + person_1 + "/" + image_1_name + "_" + image_2_name + "_" + select_code + ".png"
                        im.save(new_image_path)
                        cnt_this_person += 1
                        cnt_augument += 1
                        if cnt_augument % 100 == 0:
                            print cnt_augument
                        if cnt_this_person >= 100:
                            raise StopIteration
        except StopIteration: pass

    #inter-syn
    else:
        #print "inter"
        image_list_1 = dict_person_img[person_1]
        image_list_2 = dict_person_img[person_2]
        for select_code in select_code_list_inter:
            directory = data_path + "/inter_" + person_1 + "_" + person_2 + "_" + select_code 
            if not os.path.exists(directory):
                os.makedirs(directory)
            try:
                cnt_this_person = 0
                for i in range(len(image_list_1)):
                    for j in range(len(image_list_2)):
                        image_1 = image_list_1[i]
                        image_2 = image_list_2[j]
                        new_ = crop_to_augument(image_1, image_2, select_code, dict_img_landmark)
                        #plt.imshow(new_, cmap = plt.get_cmap('gray'))
                        #plt.show()
                        rescaled = new_.astype(np.uint8)
                        im = Image.fromarray(rescaled)
                        image_1_name = image_1.split("/")[4].split(".")[0]
                        image_2_name = image_2.split("/")[4].split(".")[0]
                        new_image_path = directory + "/" + image_1_name + "_" + image_2_name + "_" + select_code + ".png"
                        im.save(new_image_path)
                        cnt_this_person += 1
                        if cnt_this_person >= 50:
                            raise StopIteration
                        cnt_augument += 1
                        if cnt_augument % 100 == 0:
                            print cnt_augument
            except StopIteration: pass
                        
            

visited:  1
visited:  2
100
visited:  3
200
visited:  4
300
visited:  5
400
visited:  6
500
visited:  7
600
visited:  8
700
visited:  9
800
visited:  10
visited:  11
900
visited:  12
1000
visited:  13
1100
visited:  14
1200
visited:  15
1300
visited:  16
1400
visited:  17
1500
visited:  18
1600
visited:  19
visited:  20
1700
visited:  21
1800
visited:  22
1900
visited:  23
2000
visited:  24
2100
visited:  25
2200
visited:  26
2300
visited:  27
2400
visited:  28
2500
visited:  29
2600
visited:  30
visited:  31
2700
visited:  32
2800
visited:  33
2900
visited:  34
3000
visited:  35
3100
visited:  36
visited:  37
3200
visited:  38
3300
visited:  39
3400
visited:  40
3500
visited:  41
3600
visited:  42
visited:  43
3700
visited:  44
3800
visited:  45
3900
visited:  46
4000
visited:  47
4100
visited:  48
4200
visited:  49
visited:  50
4300
visited:  51
4400
visited:  52
visited:  53
4500
visited:  54
4600
visited:  55
visited:  56
4700
visited:  57
4800
visited:  58
4900
visited:  59
5000
v

KeyboardInterrupt: 

In [56]:
import random
import os
for folder in os.listdir(data_path):
    file_list = []
    folder_path = data_path + "/" + folder
    for file_ in os.listdir(folder_path):
        file_list.append(folder_path + "/" +file_)
    if len(file_list) > 100:
        delete_list = random.sample(file_list, len(file_list) - 100)
        for ele in delete_list:
            os.remove(ele)
        

In [58]:
for folder in os.listdir(data_path):
    file_list = []
    folder_path = data_path + "/" + folder
    for file_ in os.listdir(folder_path):
        file_list.append(folder_path + "/" +file_)
    if len(file_list) != 100:
        print folder + str(len(file_list))

Zinedine_Zidane66
Colin_Montgomerie15
Oswaldo_Paya65
Sourav_Ganguly65
Claudia_Pechstein35
Peter_Struck35
LeBron_James65
David_Anderson65
Thabo_Mbeki35
Nadia_Petrova65
Mark_Hurlbert65
Christine_Baumgartner65
Martin_McGuinness35
Rainer_Schuettler65
Clara_Harris65
Alan_Greenspan65
Tariq_Aziz66
David_Wells67
Woody_Allen65
Mick_Jagger65
John_Stockton65
Yasser_Arafat38
Elizabeth_Smart65
Carla_Del_Ponte65
Brad_Johnson65
Bernard_Law65
Dennis_Hastert66
Angela_Merkel65


In [26]:
while True:
    person_1 = person_list[random.randint(0, len(person_list))]
    person_2 = person_list[random.randint(0, len(person_list))]
        
    if [person_1, person_2] in visited:
        continue
    else:
        visited.append([person_1, person_2])
    print "visited's length: ", len(visited)
    
    #intra-syn
    if person_1 == person_2:
        #condition for num of pics, pending...
        image_list = dict_person_img[person_1]
        if len(image_list) <= 1 or len(image_list) >= 50:
            continue
        cnt_this_person = len(image_list)

        try:
            for i in range(len(image_list) - 1):
                for j in range(i + 1, len(image_list)):
                    image_1 = image_list[i]
                    image_2 = image_list[j]
                    for select_code in select_code_list_intra:
                        new_ = crop_to_augument(image_1, image_2, select_code, dict_img_landmark)
    #                         plt.imshow(new_, cmap = plt.get_cmap('gray'))
    #                         plt.show()
                        rescaled = new_.astype(np.uint8)
                        im = Image.fromarray(rescaled)
                        image_1_name = image_1.split("/")[4].split(".")[0]
                        image_2_name = image_2.split("/")[4].split(".")[0]
                        new_image_path = data_path + "/" + person_1 + "/" + image_1_name + "_" + image_2_name + "_" + select_code + ".png"
                        im.save(new_image_path)
                        cnt_this_person += 1
                        cnt_augument += 1
                        if cnt_augument % 100 == 0:
                            print cnt_augument
                        if cnt_this_person >= 50:
                            raise StopIteration
        except StopIteration: pass

    #inter-syn
    else:
        #print "inter"
        image_list_1 = dict_person_img[person_1]
        image_list_2 = dict_person_img[person_2]
        for select_code in select_code_list_inter:
            directory = data_path + "/inter_" + person_1 + "_" + person_2 + "_" + select_code 
            if not os.path.exists(directory):
                os.makedirs(directory)
            try:
                cnt_this_person = 0
                for i in range(len(image_list_1)):
                    for j in range(len(image_list_2)):
                        image_1 = image_list_1[i]
                        image_2 = image_list_2[j]
                        new_ = crop_to_augument(image_1, image_2, select_code, dict_img_landmark)
                        #plt.imshow(new_, cmap = plt.get_cmap('gray'))
                        #plt.show()
                        rescaled = new_.astype(np.uint8)
                        im = Image.fromarray(rescaled)
                        image_1_name = image_1.split("/")[4].split(".")[0]
                        image_2_name = image_2.split("/")[4].split(".")[0]
                        new_image_path = directory + "/" + image_1_name + "_" + image_2_name + "_" + select_code + ".png"
                        im.save(new_image_path)
                        cnt_this_person += 1
                        if cnt_this_person >= 50:
                            raise StopIteration
                        cnt_augument += 1
                        if cnt_augument % 100 == 0:
                            print cnt_augument
            except StopIteration: pass

visited's length:  1


IOError: [Errno 2] No such file or directory: './dataset/lfw_mtcnnpy_gray_picked_10_3/Rick_Pitino/Rick_Pitino_0001.png'

In [120]:
while True:
    person_1 = person_list[random.randint(0, len(person_list))]
    person_2 = person_list[random.randint(0, len(person_list))]
        
    if [person_1, person_2] in visited:
        continue
    else:
        visited.append([person_1, person_2])
    
    #intra-syn
    if person_1 == person_2:
        #condition for num of pics, pending...
        image_list = dict_person_img[person_1]
        if len(image_list) <= 1 or len(image_list) >= 50:
            continue
        cnt_this_person = len(image_list)

        try:
            for i in range(len(image_list) - 1):
                for j in range(i + 1, len(image_list)):
                    image_1 = image_list[i]
                    image_2 = image_list[j]
                    for select_code in select_code_list_intra:
                        new_ = crop_to_augument(image_1, image_2, select_code, dict_img_landmark)
    #                         plt.imshow(new_, cmap = plt.get_cmap('gray'))
    #                         plt.show()
                        rescaled = new_.astype(np.uint8)
                        im = Image.fromarray(rescaled)
                        image_1_name = image_1.split("/")[4].split(".")[0]
                        image_2_name = image_2.split("/")[4].split(".")[0]
                        new_image_path = data_path + "/" + person_1 + "/" + image_1_name + "_" + image_2_name + "_" + select_code + ".png"
                        im.save(new_image_path)
                        cnt_this_person += 1
                        cnt_augument += 1
                        if cnt_augument % 100 == 0:
                            print cnt_augument
                        if cnt_this_person >= 50:
                            raise StopIteration
        except StopIteration: pass

    #inter-syn
    else:
        #print "inter"
        image_list_1 = dict_person_img[person_1]
        image_list_2 = dict_person_img[person_2]
        for select_code in select_code_list_inter:
            directory = data_path + "/inter_" + person_1 + "_" + person_2 + "_" + select_code 
            if not os.path.exists(directory):
                os.makedirs(directory)
            try:
                cnt_this_person = 0
                for i in range(len(image_list_1)):
                    for j in range(len(image_list_2)):
                        image_1 = image_list_1[i]
                        image_2 = image_list_2[j]
                        new_ = crop_to_augument(image_1, image_2, select_code, dict_img_landmark)
                        #plt.imshow(new_, cmap = plt.get_cmap('gray'))
                        #plt.show()
                        rescaled = new_.astype(np.uint8)
                        im = Image.fromarray(rescaled)
                        image_1_name = image_1.split("/")[4].split(".")[0]
                        image_2_name = image_2.split("/")[4].split(".")[0]
                        new_image_path = directory + "/" + image_1_name + "_" + image_2_name + "_" + select_code + ".png"
                        im.save(new_image_path)
                        cnt_this_person += 1
                        if cnt_this_person >= 50:
                            raise StopIteration
                        cnt_augument += 1
                        if cnt_augument % 100 == 0:
                            print cnt_augument
            except StopIteration: pass

80300
80400
80500
80600
80700
80800
80900
81000
81100
81200
81300
81400
81500
81600
81700
81800
81900
82000
82100
82200
82300
82400
82500
82600
82700


SystemError: tile cannot extend outside image

In [121]:
while True:
    person_1 = person_list[random.randint(0, len(person_list))]
    person_2 = person_list[random.randint(0, len(person_list))]
        
    if [person_1, person_2] in visited:
        continue
    else:
        visited.append([person_1, person_2])
    
    #intra-syn
    if person_1 == person_2:
        #condition for num of pics, pending...
        image_list = dict_person_img[person_1]
        if len(image_list) <= 1 or len(image_list) >= 50:
            continue
        cnt_this_person = len(image_list)

        try:
            for i in range(len(image_list) - 1):
                for j in range(i + 1, len(image_list)):
                    image_1 = image_list[i]
                    image_2 = image_list[j]
                    for select_code in select_code_list_intra:
                        new_ = crop_to_augument(image_1, image_2, select_code, dict_img_landmark)
    #                         plt.imshow(new_, cmap = plt.get_cmap('gray'))
    #                         plt.show()
                        rescaled = new_.astype(np.uint8)
                        im = Image.fromarray(rescaled)
                        image_1_name = image_1.split("/")[4].split(".")[0]
                        image_2_name = image_2.split("/")[4].split(".")[0]
                        new_image_path = data_path + "/" + person_1 + "/" + image_1_name + "_" + image_2_name + "_" + select_code + ".png"
                        im.save(new_image_path)
                        cnt_this_person += 1
                        cnt_augument += 1
                        if cnt_augument % 100 == 0:
                            print cnt_augument
                        if cnt_this_person >= 50:
                            raise StopIteration
        except StopIteration: pass

    #inter-syn
    else:
        #print "inter"
        image_list_1 = dict_person_img[person_1]
        image_list_2 = dict_person_img[person_2]
        for select_code in select_code_list_inter:
            directory = data_path + "/inter_" + person_1 + "_" + person_2 + "_" + select_code 
            if not os.path.exists(directory):
                os.makedirs(directory)
            try:
                cnt_this_person = 0
                for i in range(len(image_list_1)):
                    for j in range(len(image_list_2)):
                        image_1 = image_list_1[i]
                        image_2 = image_list_2[j]
                        new_ = crop_to_augument(image_1, image_2, select_code, dict_img_landmark)
                        #plt.imshow(new_, cmap = plt.get_cmap('gray'))
                        #plt.show()
                        rescaled = new_.astype(np.uint8)
                        im = Image.fromarray(rescaled)
                        image_1_name = image_1.split("/")[4].split(".")[0]
                        image_2_name = image_2.split("/")[4].split(".")[0]
                        new_image_path = directory + "/" + image_1_name + "_" + image_2_name + "_" + select_code + ".png"
                        im.save(new_image_path)
                        cnt_this_person += 1
                        if cnt_this_person >= 50:
                            raise StopIteration
                        cnt_augument += 1
                        if cnt_augument % 100 == 0:
                            print cnt_augument
            except StopIteration: pass

82800
82900
83000
83100
83200
83300
83400
83500
83600
83700
83800
83900
84000
84100
84200
84300
84400
84500
84600
84700
84800
84900
85000
85100
85200
85300
85400
85500
85600
85700
85800
85900
86000
86100
86200
86300
86400
86500
86600
86700
86800
86900
87000
87100
87200
87300
87400
87500
87600
87700
87800
87900
88000
88100
88200
88300
88400
88500
88600
88700
88800
88900
89000
89100
89200
89300
89400
89500
89600
89700
89800
89900
90000
90100
90200
90300
90400
90500
90600
90700
90800
90900
91000
91100
91200
91300
91400
91500
91600
91700
91800
91900
92000
92100
92200
92300
92400
92500
92600
92700
92800
92900
93000
93100
93200
93300
93400
93500
93600
93700
93800
93900
94000
94100
94200
94300
94400
94500
94600
94700
94800
94900
95000
95100
95200
95300
95400
95500
95600
95700
95800
95900
96000
96100
96200
96300
96400
96500
96600
96700
96800
96900
97000
97100
97200
97300
97400
97500
97600
97700
97800
97900
98000
98100
98200
98300
98400
98500
98600
98700
98800
98900
99000
99100
99200
99300
9940

IOError: [Errno 28] No space left on device: 'temp.png'

In [125]:
thefile = open('visited.txt', 'w')
for item in visited:
    print "1"
    for ele in item:
        thefile.write(ele)
        thefile.write(" ")

1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
